In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pycbc.types import timeseries
from calcwf import *

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


lal.MSUN_SI != Msun
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new


# Calculating harmonics

In [4]:
# Path names to waveforms
waveform_dir = '/home/isobel.romero-shaw/public_html/PYCENTRICITY/pycentricity-plus-precession/ben_project/'
harmonic_basis_paths = [waveform_dir+f'harmonic_wf_{index}.txt' for index in [10, *np.arange(9)]]

In [3]:
# Create pycbc timeseries of waveforms with initial data cut to enforce same size/same merger time
def load_comp_wfs():
    harmonic_basis = []
    harmonic_basis_raw = [np.loadtxt(path, delimiter=',') for path in harmonic_basis_paths]
    min_len = np.min([len(raw_wf) for raw_wf in harmonic_basis_raw])
    for raw_wf in harmonic_basis_raw:
        t = raw_wf[-min_len:,0]
        hp = raw_wf[-min_len:,1]
        hc = raw_wf[-min_len:,2]
        h = timeseries.TimeSeries(hp - 1j*hc, 1/round(1/np.mean(np.diff(t))), epoch=0)
        harmonic_basis.append(h)
    return harmonic_basis

In [7]:
# Get component waveforms for basis 
# Will use this function in edited get_h() to get harmonics
def gen_component_wfs_seobnrpe(f_low, e, M, q, n, sample_rate, approximant, normalisation, phase, f_match):

    # Make sure chosen parameters are those used to generate these component waveforms
    assert f_low == 10
    assert e == 0.073
    assert q == 2
    assert M == chirp2total(23.97979764, 2)
    assert n == 10
    assert sample_rate == 2048
    assert phase == 0
    
    # Load in all component waveforms
    comp_wfs = load_comp_wfs()

    # Taper all waveforms
    comp_wfs = [taper_wf(h) for h in comp_wfs]

    # Correct phase of all beyond first
    overlaps = [overlap_cplx_wfs(h, comp_wfs[0], f_low, f_match=f_match) for h in comp_wfs[1:]]
    phase_angles = -np.angle(overlaps)/2
    comp_wfs[1:] = [comp_wfs[i+1]*np.exp(2*1j*phase_angles[i]) for i in range(9)]

    # Normalise if requested
    if normalisation:

        # Get normalisation from first waveform
        h.resize(ceiltwo(len(h))) 
        psd = gen_psd(h, f_low)
        sigma_0 = sigma(h.real(), psd=psd, low_frequency_cutoff=f_match)

        # Normalise subsequent waveforms
        sigma_hs = [sigma(h.real(), psd=psd, low_frequency_cutoff=f_match) for h in comp_wfs]
        comp_wfs[1:] = [comp_wfs[i+1]*sigma_0/sigma_h[i] for i in range(9)]
        
    return comp_wfs

# Edited get_h() to call above component wf function
def get_h_seobnrpe(coeffs, f_low, e, M, q, sample_rate, approximant='SEOBNRPE', f_match=20, subsample_interpolation=True, 
                   GS_normalisation=True, comp_normalisation=False, comp_phase=0, return_ovlps=False):
    """
    Generates a overall h waveform, h_1,...h_n, and s_1,...,s_n.

    Parameters:
        coeffs: List containing coefficients of h_1,...,h_n.
        f_low: Starting frequency.
        e: Eccentricity.
        M: Total mass.
        q: Mass ratio.
        sample_rate: Sample rate of waveform.
        approximant: Approximant to use.
        f_match: Low frequency cutoff to use.
        subsample_interpolation: Whether to use subsample interpolation.
        GS_normalisation: Whether to perform Grant-Schmidt orthogonalisaton to ensure (hj|hm) = 0 for j!=m.
        comp_normalisation: Whether to normalise s_1,...,s_n components to ensure (sj|sj) is constant.
        comp_phase: Initial phase of s_1,...,s_n components.
        return_ovlps: Whether to return overlaps between all unorthogonalised harmonics.
        
    Returns:
        All waveform components and combinations: h, h1, ..., h_n, s_1, ..., s_n
    """

    # Other approximants are deprecated
    assert approximant == 'SEOBNRPE'

    # Gets (normalised) components which make up overall waveform
    component_wfs = gen_component_wfs_seobnrpe(f_low, e, M, q, len(coeffs), sample_rate, approximant, comp_normalisation, comp_phase, f_match)

    # Calculate overall waveform and components in time domain
    wfs, ovlps, ovlps_perp = get_h_TD(f_low, coeffs, component_wfs, GS_normalisation, f_match, return_ovlps=return_ovlps)

    if return_ovlps:
        return wfs, ovlps, ovlps_perp
    else:    
        return wfs

In [5]:
# Generate harmonics
all_wfs = get_h_seobnrpe([1]*10, 10, 0.073, chirp2total(23.97979764, 2), 2, 2048)
h_wfs = all_wfs[1:11]

# Formatting of trial waveforms

In [11]:
# Load json of trial waveforms
def load_trial_wfs(path):

    # Loads in with 'raw' formatting
    with open(path) as f:
        raw_wfs = json.load(f)

    print(np.shape(raw_wfs['wfs']))

In [12]:
trial_wfs = load_trial_wfs(waveform_dir+'trial_wfs.json')

(32,)


# Calculating matches

# Estimating eccentricity